In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

print("Slicing dataset")

dir_n = "/home/rodr/buffalo/rodr/Datos/NSRDB/"
dir_p = "/datos/rodr/temp/recurso_solar/duck_curve/"
file_p = f"{dir_p}conus_points.csv"
file_d = f"{dir_n}nsrdb_conus_irradiance_2022.h5"
file_r = f"{dir_p}nsrdb_conus_irradiance_2022_crop.nc"
points = pd.read_csv(file_p, index_col = 0).sort_index().index
ds = xr.open_dataset(file_d,
    drop_variables = ["dni", "fill_flag", "time_index"], chunks = "auto"
    ).rename_dims({"phony_dim_1": "points", "phony_dim_0": "time_index"})

for j in range(0, np.ceil(points.shape[0]/1000).astype(int) + 1):
    print()
    print(j*1000)

    df = ds.isel(time_index = slice(0, None, 12), points = points[0 + 1000*j]
        ).to_dataframe().astype(np.int16)

    for i in range(1 + 1000*j,1000 + 1000*j):
        if i%100==0: print(i, end = ", ")
        p = points[i]
        df = pd.concat([df, ds.isel(time_index = slice(0, None, 12),
            points = p).to_dataframe().astype(np.int16)])
        
    df["points"] = np.repeat(points[0+j*1000:1000+j*1000],
        ds["time_index"].shape[0]/12)
    df.set_index("points", append = True).to_xarray().to_netcdf(
        f"{dir_p}points/points_{j}.nc")

1100
1200
1300
1400
1500
1600
1700
1800
1900
2000


In [77]:
np.ceil(points.shape[0]/1000).astype(int)

87

In [23]:
# Genera la tabla de puntos para el estudio de la curva de pato

import h5pyd
import pandas    as pd
import numpy     as np
import geopandas as gpd
import xarray    as xr
import xesmf     as xe

print("Generando tabla de puntos")
print()

# Descargamos malla con HSDS
print("Descargando malla de CONUS")
with h5pyd.File("/nrel/nsrdb/conus/nsrdb_conus_2022.h5") as f:
    print(f['time_index'])

Generando tabla de puntos

Descargando malla de CONUS
<HDF5 dataset "time_index": shape (105120,), type "|S25">


In [39]:
dir_p = "/datos/rodr/temp/recurso_solar/duck_curve/"
file_p = f"{dir_p}conus_points.csv"
points = pd.read_csv(file_p, index_col = 0).sort_index().index

In [45]:
points[0:100]

Index([13653, 13656, 13659, 13662, 13664, 14053, 14056, 14059, 14062, 14065,
       14068, 14071, 14074, 14515, 14518, 14521, 14524, 14527, 14530, 14533,
       14536, 14539, 15088, 15091, 15094, 15097, 15102, 15105, 15108, 15111,
       15114, 15117, 15120, 15123, 15126, 15129, 15132, 15793, 15796, 15799,
       15802, 15805, 15808, 15813, 15816, 15819, 15822, 15825, 15828, 15832,
       15835, 15838, 15841, 15844, 15847, 15850, 15853, 15856, 15859, 15862,
       16695, 16698, 16701, 16704, 16707, 16710, 16713, 16716, 16719, 16722,
       16725, 16728, 16731, 16734, 16737, 16740, 16743, 16746, 16749, 16752,
       16755, 16758, 16761, 16764, 16767, 16770, 16773, 16776, 16779, 16782,
       16785, 16788, 16791, 17758, 17761, 17764, 17767, 17770, 17773, 17776],
      dtype='int64')

In [11]:
import h5py
import pandas as pd

dir_n = "/home/rodr/buffalo/rodr/Datos/NSRDB/"
dir_p = "/datos/rodr/temp/recurso_solar/duck_curve/"
file_p = f"{dir_p}conus_points.csv"
file_d = f"{dir_n}nsrdb_conus_irradiance_2022.h5"
file_r = f"{dir_n}nsrdb_conus_irradiance_2022_crop.h5"

print("Starting partial copy of dataset")
print()

print("Loading points list")
print()
points = pd.read_csv(file_p, index_col = 0).sort_index()

Starting partial copy of dataset

Loading points list



In [19]:
import xarray as xr
import numpy as np

file_n = "/home/rodr/buffalo/rodr/Datos/NSRDB/nsrdb_conus_irradiance_2022.h5"
ds = xr.open_dataset(file_n, chunks = "auto")

In [21]:
ds

<xarray.Dataset> Size: 10TB
Dimensions:     (phony_dim_0: 105120, phony_dim_1: 2842719)
Dimensions without coordinates: phony_dim_0, phony_dim_1
Data variables:
    dhi         (phony_dim_0, phony_dim_1) float64 2TB dask.array<chunksize=(8000, 2000), meta=np.ndarray>
    dni         (phony_dim_0, phony_dim_1) float64 2TB dask.array<chunksize=(8000, 2000), meta=np.ndarray>
    fill_flag   (phony_dim_0, phony_dim_1) float64 2TB dask.array<chunksize=(4000, 2000), meta=np.ndarray>
    ghi         (phony_dim_0, phony_dim_1) float64 2TB dask.array<chunksize=(8000, 2000), meta=np.ndarray>
    time_index  (phony_dim_0) <U25 11MB dask.array<chunksize=(105120,), meta=np.ndarray>
Attributes:
    version:  4.0.0

In [ ]:
ds.drop_vars(["dni", "fill_flag"]).isel({"phony_dim_0": slice(0, None, 12)}
    ).astype(np.int16).nbytes/1024**3

92.76800908148289

In [ ]:
ds.astype.nbytes/1024**3

8905.737095475197

In [13]:
# Calcula los elementos físicos de la radiación solar que dependen de la
# ubicación y del tiempo.

import pandas as pd
import numpy as np
import xarray as xr

# Funciones trigonométricas.
def sin(x) : return np.sin(np.radians(x))
def cos(x) : return np.cos(np.radians(x))
def tan(x) : return np.tan(np.radians(x))
def asin(x): return np.arcsin(x) * 180/np.pi
def acos(x): return np.arccos(x) * 180/np.pi
def atan(x): return np.arctan(x) * 180/np.pi

print("Seguimiento solar")
print()

# Casos a estudiar
cases = [ "south_no_track", "west_no_track", "east_no_track",
    "1_track", "2_track", "bifacial_vertical_west_main",
    "bifacial_vertical_east_main", "bifacial_vertical_west_back",
    "bifacial_vertical_east_back", "bifacial_south_back" ]
# Variables fotovoltaicas por caso
# Inclinación
track_tilt       =   [ f"{x}_Tilt"               for x in cases ]
# Azimuth
track_azimuth    =   [ f"{x}_Azimuth"            for x in cases ]
# Ángulo entre el panel y el sol, Angle of Incidence
track_AOI        =   [ f"{x}_Angle_of_Incidence" for x in cases ]

print("Cargando archivos")
dir_d = "/datos/rodr/temp/recurso_solar/duck_curve/"
ds = xr.open_dataset(f"{dir_d}full_disc_physical_solar_2022.nc",
    chunks = {"time": 24})[["Zenith_Angle", "Azimuth_Angle"]
    + track_tilt + track_azimuth]

# Ángulo de incidencia y valores de método de Pérez
print("Calculando AOI y Pérez a")
for i in range(len(cases)):
    print(f"Calculando {cases[i]}")
    # Ángulo entre el panel y el sol, Angle of Incidence
    ds[track_AOI[i]] = (cos(ds["Zenith_Angle"])*cos(ds[track_tilt[i]])
        + sin(ds["Zenith_Angle"])*sin(ds[track_tilt[i]])
        *cos(ds["Azimuth_Angle"]-ds[track_azimuth[i]]
        ).clip(-1, 1)).astype(np.float32)
    ds[track_AOI[i]] = acos(ds[track_AOI[i]])
    ds = ds.drop_vars([track_tilt[i], track_azimuth[i]])
    print(ds.nbytes/1024**3)

ds = ds.drop_vars(["Zenith_Angle", "Azimuth_Angle"])
print(ds.nbytes/1024**3)

Seguimiento solar

Cargando archivos
Calculando AOI y Pérez a
Calculando south_no_track
16.48949810117483
Calculando west_no_track
18.84428631514311
Calculando east_no_track
21.199074529111385
Calculando 1_track
18.843748562037945
Calculando 2_track
16.488422594964504
Calculando bifacial_vertical_west_main
18.843748547136784
Calculando bifacial_vertical_east_main
21.199074499309063
Calculando bifacial_vertical_west_back
23.554400451481342
Calculando bifacial_vertical_east_back
25.90972640365362
Calculando bifacial_south_back
28.2645146176219
23.553862683475018


In [14]:
ds

<xarray.Dataset> Size: 25GB
Dimensions:                                         (points: 72175, time: 8760)
Coordinates:
  * points                                          (points) int64 577kB 1881...
  * time                                            (time) datetime64[ns] 70kB ...
Data variables:
    south_no_track_Angle_of_Incidence               (points, time) float32 3GB dask.array<chunksize=(72175, 24), meta=np.ndarray>
    west_no_track_Angle_of_Incidence                (points, time) float32 3GB dask.array<chunksize=(72175, 24), meta=np.ndarray>
    east_no_track_Angle_of_Incidence                (points, time) float32 3GB dask.array<chunksize=(72175, 24), meta=np.ndarray>
    1_track_Angle_of_Incidence                      (points, time) float32 3GB dask.array<chunksize=(72175, 24), meta=np.ndarray>
    2_track_Angle_of_Incidence                      (points, time) float32 3GB dask.array<chunksize=(72175, 24), meta=np.ndarray>
    bifacial_vertical_west_main_Angle_of_Incidence  (points, time) float32 3GB dask.array<chunksize=(72175, 24), meta=np.ndarray>
    bifacial_vertical_east_main_Angle_of_Incidence  (points, time) float32 3GB dask.array<chunksize=(72175, 24), meta=np.ndarray>
    bifacial_vertical_west_back_Angle_of_Incidence  (points, time) float32 3GB dask.array<chunksize=(72175, 24), meta=np.ndarray>
    bifacial_vertical_east_back_Angle_of_Incidence  (points, time) float32 3GB dask.array<chunksize=(72175, 24), meta=np.ndarray>
    bifacial_south_back_Angle_of_Incidence          (points, time) float32 3GB dask.array<chunksize=(72175, 24), meta=np.ndarray>

In [10]:
ds.nbytes/1024**3

49.467471508309245

In [1]:
 # Funciones trigonométricas.
def sin(x) : return np.sin(np.radians(x))
def cos(x) : return np.cos(np.radians(x))
def tan(x) : return np.tan(np.radians(x))
def asin(x): return np.arcsin(x) * 180/np.pi
def acos(x): return np.arccos(x) * 180/np.pi
def atan(x): return np.arctan(x) * 180/np.pi


In [7]:
TZ = ds["timezone"]
lat = ds["latitude"]
lon = ds["longitude"]
a = ((sin(ds["Declination"])
    - cos(ds["Zenith_Angle"])*sin(lat))
    / (sin(ds["Zenith_Angle"])*cos(lat)))

In [2]:
import pandas as pd
import numpy as np
import xarray as xr

In [22]:
b =sin(ds["Zenith_Angle"])*cos(lat)

In [27]:
(ds["Hour_Angle"]==0).sum()

<xarray.DataArray 'Hour_Angle' ()> Size: 8B
array(0)

In [24]:
b.min()

<xarray.DataArray ()> Size: 8B
array(-0.00016722)

In [1]:
import h5pyd
import pandas as pd
import numpy as np
import xarray as xr

In [2]:
# Funciones trigonométricas.
def sin(x) : return np.sin(np.radians(x))
def cos(x) : return np.cos(np.radians(x))
def tan(x) : return np.tan(np.radians(x))
def asin(x): return np.arcsin(x) * 180/np.pi
def acos(x): return np.arccos(x) * 180/np.pi
def atan(x): return np.arctan(x) * 180/np.pi

# Funciones para claves de regiones eléctricas

# Clave a número
def NUM_REGION(x):
    if   x == "BCA"  : y = 1
    elif x == "BCS"  : y = 2
    elif x == "CEN"  : y = 3
    elif x == "NES"  : y = 4
    elif x == "NOR"  : y = 5
    elif x == "NTE"  : y = 6
    elif x == "OCC"  : y = 7
    elif x == "ORI"  : y = 8
    elif x == "PEN"  : y = 9
    elif x == "PGE"  : y = 10
    elif x == "SCE"  : y = 11
    elif x == "SDGE" : y = 12
    elif x == "VEA"  : y = 13
    elif x == "COAST": y = 14
    elif x == "EAST" : y = 15
    elif x == "FWEST": y = 16
    elif x == "NORTH": y = 17
    elif x == "NCENT": y = 18
    elif x == "SOUTH": y = 19
    elif x == "SCENT": y = 20
    elif x == "WEST" : y = 21
    elif x == "LON"  : y = 22
    elif x == "LAT"  : y = 23
    elif x == "TOTAL": y = 24
    return y

# Número a clave
def NUM_REGION_r(x):
    if   x == 1 : y = "BCA"
    elif x == 2 : y = "BCS"
    elif x == 3 : y = "CEN"
    elif x == 4 : y = "NES"
    elif x == 5 : y = "NOR"
    elif x == 6 : y = "NTE"
    elif x == 7 : y = "OCC"
    elif x == 8 : y = "ORI"
    elif x == 9 : y = "PEN"
    elif x == 10: y = "PGE"
    elif x == 11: y = "SCE"
    elif x == 12: y = "SDGE"
    elif x == 13: y = "VEA"
    elif x == 14: y = "COAST"
    elif x == 15: y = "EAST"
    elif x == 16: y = "FWEST"
    elif x == 17: y = "NORTH"
    elif x == 18: y = "NCENT"
    elif x == 19: y = "SOUTH"
    elif x == 20: y = "SCENT"
    elif x == 21: y = "WEST"
    elif x == 22: y = "LON"
    elif x == 23: y = "LAT"
    elif x == 24: y = "TOTAL"
    return y

# Número a nombre
def NAME_REGION_r(x):
    if   x == 1 : y = "Baja California"
    elif x == 2 : y = "Baja California Sur"
    elif x == 3 : y = "Centro"
    elif x == 4 : y = "Noreste"
    elif x == 5 : y = "Noroeste"
    elif x == 6 : y = "Norte"
    elif x == 7 : y = "Occidente"
    elif x == 8 : y = "Oriente"
    elif x == 9 : y = "Peninsular"
    elif x == 10: y = "Pacific G&E"
    elif x == 11: y = "Southern Cal Edison"
    elif x == 12: y = "San Diego G&E"
    elif x == 13: y = "Valley Electric Assn"
    elif x == 14: y = "Coast"
    elif x == 15: y = "East"
    elif x == 16: y = "Far West"
    elif x == 17: y = "North"
    elif x == 18: y = "North Central"
    elif x == 19: y = "South"
    elif x == 20: y = "South Central"
    elif x == 21: y = "West"
    elif x == 22: y = "Longitudinal"
    elif x == 23: y = "Latitudinal"
    elif x == 24: y = "Total"
    return y

# Número a huso horario
def TIME_REGION_r(x):
    if   x == 1 : y = -8
    elif x == 2 : y = -7
    elif x == 3 : y = -6
    elif x == 4 : y = -6
    elif x == 5 : y = -7
    elif x == 6 : y = -6
    elif x == 7 : y = -6
    elif x == 8 : y = -6
    elif x == 9 : y = -6
    elif x == 10: y = -8
    elif x == 11: y = -8
    elif x == 12: y = -8
    elif x == 13: y = -8
    elif x == 14: y = -6
    elif x == 15: y = -6
    elif x == 16: y = -6
    elif x == 17: y = -6
    elif x == 18: y = -6
    elif x == 19: y = -6
    elif x == 20: y = -6
    elif x == 21: y = -6
    return y

# Número a diferencia de huso horario
def DELTA_REGION_r(x): return -( TIME_REGION_r(x) + 8 )

In [8]:
dir_d = "/home/rodr/temp/recurso_solar/duck_curve/"
points = pd.read_csv(f"{dir_d}/conus_points.csv", index_col = 0)
points = points[points["REGION"].notnull()]
points["REGION"] = points["REGION"].astype(np.int16)
ds = points[["latitude", "longitude", "elevation", "timezone", "REGION",
    "potential_solar_park_zones", "built_surface"]
    ].to_xarray().rename({"index": "points"})
ds["time"] = pd.date_range("2022-01-01", "2023-01-01", freq = "1h")[:-1]
ds = ds.chunk(time = 24)
ds["Pressure"] = (1013.25 * (1 - 0.0065/288.15*ds["elevation"]) ** 5.25588
    ).astype(np.float32)
ds = ds.drop_vars("elevation")
ds

<xarray.Dataset> Size: 4MB
Dimensions:                     (points: 72175, time: 8760)
Coordinates:
  * points                      (points) int64 577kB 1881061 1884568 ... 21809
  * time                        (time) datetime64[ns] 70kB 2022-01-01 ... 202...
Data variables:
    latitude                    (points) float64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    longitude                   (points) float64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    timezone                    (points) float64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    REGION                      (points) int16 144kB dask.array<chunksize=(72175,), meta=np.ndarray>
    potential_solar_park_zones  (points) int64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    built_surface               (points) float64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    Pressure                    (points) float32 289kB dask.array<chunksize=(72175,), meta=np.ndarray>

In [9]:
# Convertimos a fecha.
ds["hour"] = (ds["time"].dt.hour + ds["time"].dt.minute/60).astype(np.float32)
ds["dayofyear"] = ds["time"].dt.dayofyear.astype(np.int16)

TZ = ds["timezone"]
lat = ds["latitude"]
lon = ds["longitude"]

# Eccentric anomaly of the earth in its orbit around the sun.
ds["Day_Angle"] = (6.283185 * ( ds["dayofyear"] - 1 ) / 365).astype(np.float32)
# Declinación.
ds["Declination"] = ( ( 0.006918 - 0.399912 * np.cos(ds["Day_Angle"])
    + 0.070257*np.sin(ds["Day_Angle"])
    - 0.006758*np.cos(2*ds["Day_Angle"])
    + 0.000907*np.sin(2*ds["Day_Angle"])
    - 0.002697*np.cos(3*ds["Day_Angle"])
    + 0.00148*np.sin(3*ds["Day_Angle"]) ) * 180/np.pi ).astype(np.float32)
# Ecuación del tiempo.
ds["Time_Equation"] = ( ( 0.000075 + 0.001868*np.cos(ds["Day_Angle"])
    - 0.032077*np.sin(ds["Day_Angle"])
    - 0.014615*np.cos(2*ds["Day_Angle"])
    -0.040849*np.sin(2*ds["Day_Angle"])) * 229.18 ).astype(np.float32)
# Longitud del punto subsolar.
ds["lon_subs"] = -15 * ( ds["hour"] - TZ
    + ds["Time_Equation"]/60 ).astype(np.float32)
# Ángulo horario.
ds["Hour_Angle"] = ( 15 * ( ds["hour"] - 12
    - ds["Time_Equation"]/60 + ((lon-TZ*15)*4)/60 ) ).astype(np.float32)
ds = ds.drop_vars(["Time_Equation"])
# Posiciones del analema solar.
ds["Sz"] = ( sin(lat)*sin(ds["Declination"])
    - cos(lat)*cos(ds["Declination"])
    *cos(ds["lon_subs"]-lon) ).astype(np.float32)
ds = ds.drop_vars(["lon_subs"])
# Ángulo del cénit solar.
ds["Zenith_Angle"] = acos(ds["Sz"]).astype(np.float32)
ds = ds.drop_vars("Sz")
# Ángulo acimutal solar.
ds["Azimuth_Angle"] = acos( ( sin(ds["Declination"])
    - cos(ds["Zenith_Angle"])*sin(lat) )
    / ( sin(ds["Zenith_Angle"])*cos(lat) ) ).astype(np.float32)
ds["Azimuth_Angle"] = ds["Azimuth_Angle"].where(
    ds["Hour_Angle"] < 0, 360 - ds["Azimuth_Angle"] )
ds = ds.drop_vars(["Declination", "Hour_Angle"])
# Masa de aire.
ds["Air_Mass"] = ( 1/(cos(ds["Zenith_Angle"])
    + 0.15/(93.885 - ds["Zenith_Angle"])**1.253 )
    * ds["Pressure"]/1013.25 ).astype(np.float32)
ds["Air_Mass"] = ds["Air_Mass"].where( ds["Zenith_Angle"] < 85.5, 0 )
ds = ds.drop_vars(["Day_Angle", "Pressure"])

In [6]:
# Casos de orientación de sistemas fotovoltaico

# Casos a estudiar
cases = [ "south_no_track", "west_no_track", "east_no_track",
    "1_track", "2_track", "bifacial_vertical_west_main",
    "bifacial_vertical_east_main", "bifacial_vertical_west_back",
    "bifacial_vertical_east_back", "bifacial_south_back" ]
# Variables fotovoltaicas por caso
# Inclinación
track_tilt       =   [ f"{x}_Tilt"               for x in cases ]
# Azimuth
track_azimuth    =   [ f"{x}_Azimuth"            for x in cases ]
# Ángulo entre el panel y el sol, Angle of Incidence
track_AOI        =   [ f"{x}_Angle_of_Incidence" for x in cases ]
# Radiación incidente en el panel [W/m^2], Plane of Array Irradiace
track_POA        =   [ f"{x}_POA"                for x in cases ]
# Producción fotovoltaica por kilowatt de capacidad [W/kWp]
track_P_mp       = ( [ f"{x}_P_mp"               for x in cases ]
    + [ "bifacial_vertical_west_P_mp",
        "bifacial_vertical_east_P_mp",
        "bifacial_south_P_mp" ] )
# Producción para cada caso
prod_n           = track_P_mp[0:5] + track_P_mp[10:]
# Factor bifacial
P_bf = [ 1, 1, 1, 1, 1, 1, 1, 0.7, 0.7, 0.7 ]

# Ángulos de orientación de los sistemas
# Orientación del seguidor de un eje
# Asumimos inclinación de 0 grados
azimuth_tracker = 180
# south_no_track
ds[track_azimuth[0]] = 180
ds[track_tilt[0]   ] = lat
# west_no_track
ds[track_azimuth[1]] = 270
ds[track_tilt[1]   ] = lat
# east_no_track
ds[track_azimuth[2]] = 90
ds[track_tilt[2]   ] = lat
# 1_track
ds[track_tilt[3]   ] = np.abs( atan( tan(ds["Zenith_Angle"])
    * sin(ds["Azimuth_Angle"] - azimuth_tracker) ) ).astype(np.float32)
ds[track_azimuth[3]] = 90
ds[track_azimuth[3]] = ds["1_track_Azimuth"
    ].where(ds["Azimuth_Angle"]<180, 270).astype(np.float32)
# 2_track
ds[track_tilt[4]   ] = ds["Zenith_Angle"]
ds[track_azimuth[4]] = ds["Azimuth_Angle"]
# bifacial_vertical_west_main
ds[track_tilt[5]   ] = 90
ds[track_azimuth[5]] = 270
# bifacial_vertical_east_main
ds[track_tilt[6]   ] = 90
ds[track_azimuth[6]] = 90
# bifacial_vertical_west_back
ds[track_tilt[7]   ] = 90
ds[track_azimuth[7]] = 90
# bifacial_vertical_east_back
ds[track_tilt[8]   ] = 90
ds[track_azimuth[8]] = 270
# bifacial_south_back
ds[track_tilt[9]   ] = 90 + lat
ds[track_azimuth[9]] = 0

In [7]:
print(ds.nbytes/1024**3)
ds = ds.transpose("points", "time").chunk(time = 24)
ds

16.492907969281077


<xarray.Dataset> Size: 18GB
Dimensions:                              (points: 72175, time: 8760)
Coordinates:
  * points                               (points) int64 577kB 1881061 ... 21809
  * time                                 (time) datetime64[ns] 70kB 2022-01-0...
Data variables: (12/31)
    latitude                             (points) float64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    longitude                            (points) float64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    timezone                             (points) float64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    REGION                               (points) int16 144kB dask.array<chunksize=(72175,), meta=np.ndarray>
    potential_solar_park_zones           (points) int64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    built_surface                        (points) float64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    ...                                   ...
    bifacial_vertical_west_back_Tilt     int64 8B 90
    bifacial_vertical_west_back_Azimuth  int64 8B 90
    bifacial_vertical_east_back_Tilt     int64 8B 90
    bifacial_vertical_east_back_Azimuth  int64 8B 270
    bifacial_south_back_Tilt             (points) float64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    bifacial_south_back_Azimuth          int64 8B 0